# Data manipulation on the key_events and scraping of the adopted texts

## 1. Library and data import

In [1]:
import pandas as pd
import ast
import requests
from bs4 import BeautifulSoup
import math
from collections import OrderedDict
from datetime import datetime

In [2]:
# Load the data
df = pd.read_csv('final_sample_cod_legislative_scrape_2025.csv')
df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
pd.set_option('display.max_columns', None)

In [3]:
df

,url,reference,title,subjects,key_players,key_events,documentation_gateway,transparency,final_act,is_legislative_proposal,Procedure reference,Procedure type,Procedure subtype,Legislative instrument,Amendments and repeals,Legal basis,Stage reached in procedure,Committee dossier,Mandatory consultation of other institutions,Other legal basis,legislative_proposal_url,legislative_proposal_text
0,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0255(COD),Justice programme 2028-2034,7.40.02 Judicial cooperation in civil and comm...,{'European Parliament': [{'Joint committee res...,"{'03/09/2025': [{'Date': '03/09/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN,False,2025/0255(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,"[{'text': 'Repealing Regulation 2021/693'}, {'...",Rules of Procedure EP 58 Treaty on the Functio...,Awaiting committee decision,CJ03/10/03804,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 3.9.2025 COM(2..."
1,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0405(COD),Placing on the market of genetically modified ...,"3.10.09.06 Agro-genetics, GMOs, 4.60.02 Consum...",{'European Parliament': [{'Joint committee res...,"{'16/12/2025': [{'Date': '16/12/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN,True,2025/0405(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Directive,"[{'text': 'Amending Directive 2001/18/EC'}, {'...",Treaty on the Functioning of the EU TFEU 114 T...,Preparatory phase in Parliament,NaN,[{'text': 'European Economic and Social Commit...,NaN,NaN,NaN
2,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0540(COD),"Union support for asylum, migration and integr...",7.10 Free movement and integration of third-co...,{'European Parliament': [{'Committee responsib...,"{'16/07/2025': [{'Date': '16/07/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...",{'Other Members': [{'Name': [{'text': 'ASENS L...,NaN,True,2025/0540(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,NaN,Rules of Procedure EP 58 Treaty on the Functio...,Awaiting committee decision,LIBE/10/03652,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 16.7.2025 COM(..."
3,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0550(COD),'AgoraEU' programme 2028–2034,"1.20 Citizen's rights, 4.10.03 Child protectio...",{'European Parliament': [{'Joint committee res...,"{'16/07/2025': [{'Date': '16/07/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",NaN,True,2025/0550(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,"[{'text': 'Repealing Regulation 2021/818'}, {'...",Rules of Procedure EP 58 Rules of Procedure EP...,Awaiting committee decision,CJ65/10/03702,[{'text': 'European Economic and Social Commit...,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 16.7.2025 COM(..."
4,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0227(COD),Global Europe,"6.20 Common commercial policy in general, 8.70...",{'European Parliament': [{'Joint committee res...,"{'16/07/2025': [{'Date': '16/07/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN,True,2025/0227(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,NaN,Rules of Procedure EP 58 Rules of Procedure EP...,Awaiting committee decision,CJ19/10/03697,"[{'text': 'European Court of Auditors', 'href'...",NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 16.7.2025 COM(..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0232(COD),Protection of workers from the risks related t...,"4.15.15 Health and safety at work, occupationa...",{'Eur

## 2. Reordering the key_events dictionaries for data manipulation

In [ ]:
def reorder_event_dict(events_dict):
    """
    Converts a dictionary indexed by date into a dictionary indexed by event,
    sorted chronologically.
    """
    # 1. Flattening
    temp_list = []
    for date_str, event_list in events_dict.items():
        for e in event_list:
            temp_list.append({
                "Date": e.get("Date", date_str),
                "Event": e.get("Event", ""),
                "Reference": e.get("Reference", ""),
                "Summary": e.get("Summary", "")
            })

    # 2. Chronological sorting
    # The list is sorted in place for efficiency
    temp_list.sort(key=lambda x: datetime.strptime(x["Date"], "%d/%m/%Y"))

    # 3. Reconstructing a standard dict (order will be preserved)
    ordered = {}
    for e in temp_list:
        # Important : If two events have the same name, the last one wins.
        ordered[e["Event"]] = {
            "Date": e["Date"],
            "Reference": e["Reference"],
            "Summary": e["Summary"]
        }

    return ordered

In [5]:
# Function to apply the reorder_event_dict safely (without errors if the row is empty)
def safe_reorder(x):
    # Handle empty values
    if pd.isna(x) or (isinstance(x, float) and math.isnan(x)):
        return {} 
    # If not empty, reorder the dictionnary by date
    return reorder_event_dict(ast.literal_eval(x))

In [6]:
# Reorder the key_events by date
df['key_events'] = df['key_events'].apply(safe_reorder)

In [7]:
# Checking if the output meets the expected format
print(df['key_events'][100])

{'Legislative proposal published': {'Date': '16/07/2025', 'Reference': [{'text': 'COM(2025)0559', 'href': 'https://www.europarl.europa.eu/RegData/docs_autres_institutions/commission_europeenne/com/2025/0559/COM_COM(2025)0559_EN.pdf'}, {'text': '', 'href': 'https://eur-lex.europa.eu/smartapi/cgi/sga_doc?smartapi!celexplus!prod!DocNumber&lg=EN&type_doc=COMfinal&an_doc=2025&nu_doc=0559'}], 'Summary': [{'text': 'Summary', 'href': '/oeil/en/document-summary?id=1830670'}]}, 'Committee referral announced in Parliament, 1st reading': {'Date': '23/10/2025', 'Reference': '', 'Summary': ''}}


## 3. Extracting the url of the decisions by parliament

In [8]:
# Function to extract the URLs of the decisions taken by the Parliament for a given procedure
def extract_url_decisions(events_dict):
    """
    Extracts the href URLs of events whose name begins with 'Decision by Parliament'.
    """
    # Initialize an empty list to store URLs of Parliament decisions
    list_decisions = []

    # Loop through each event name and its associated information in the events dictionary
    for event_name, event_info in events_dict.items():
        # Check if the event name starts with 'Decision by Parliament'
        if event_name.startswith('Decision by Parliament'):
            # Extract the 'Reference' key, which may contain a list of links
            references = event_info.get('Reference', [])
            # Ensure the references field is a list
            if isinstance(references, list):
                # Loop through each reference in the list
                for ref in references:
                    # Extract the 'href' value (the actual link)
                    href = ref.get('href')
                    # If the href exists, append it to the list of decisions
                    if href:
                        list_decisions.append(href)
                        
    # Return the list of decision URLs, or None if the list is empty
    return list_decisions if list_decisions != [] else None  


In [9]:
df['decisions_url_list'] = df['key_events'].apply(extract_url_decisions)

In [10]:
print(f"{len(df[df['decisions_url_list'].notna()])} non-empty 'decisions_url_list' rows on a total of {len(df)} rows")

30 non-empty 'decisions_url_list' rows on a total of 119 rows


In [11]:
# Initialize an empty list to store indices of procedures with multiple Parliament decisions
index_list = []

# Iterate over each row index in the DataFrame
for idx in df.index:
    # Check if the 'decisions_url_list' column is not None for this row
    if df['decisions_url_list'][idx] is not None:
        # Retrieve the list of decision URLs for this procedure
        list_ = df['decisions_url_list'][idx]
        # Calculate the number of decision URLs
        length = len(list_)
        # If there is more than one decision URL, record the index of this procedure
        if length > 1:
            index_list.append(idx)           

# Print the list of indices for procedures having multiple 'Decision by Parliament' events
print(f"List of procedures having multiple Decisions by Parliament: {index_list}")

# Print the total number of such procedures
print(f"Number of procedures having multiple Decisions by Parliament: {len(index_list)}")

List of procedures having multiple Decisions by Parliament: []
Number of procedures having multiple Decisions by Parliament: 0


## 4. Scrape the Texts adopted

In [12]:
def get_resolution_text(url):
    response = requests.get(url)
    response.encoding = "utf-8"
    soup = BeautifulSoup(response.text, "html.parser")

    # Locate the start of the resolution
    title1 = soup.find("a", {"name": "title1"})
    if not title1:
        return "No resolution section (title1) found in this page"

    # Extract the table containing the resolution body
    resolution_table = title1.find_next("table", class_="doc_box_header")
    if not resolution_table:
        return "Resolution table not found"

    # Extract the title from the first <td>
    first_td = resolution_table.find("td", align="left")
    title_text = first_td.get_text(" ", strip=True) if first_td else ""

    # Collect paragraphs of the resolution
    resolution_text_parts = [title_text]  # start with title
    for p in resolution_table.find_all("p"):
        text = p.get_text(" ", strip=True)
        resolution_text_parts.append(text)

    resolution_text = "\n".join(resolution_text_parts)
    return resolution_text

In [13]:
def get_position_text(url):
    response = requests.get(url)
    response.encoding = "utf-8"
    soup = BeautifulSoup(response.text, "html.parser")

    # Locate the start of the resolution
    title1 = soup.find("a", {"name": "title2"})
    if not title1:
        return "No resolution section (title2) found in this page"

    # Extract the table containing the resolution body
    position_table = title1.find_next("table", class_="doc_box_header")
    if not position_table:
        return "Resolution table not found"
    
    # Extract the title from the first <td>
    first_td = position_table.find("td", align="left")
    title_text = first_td.get_text(" ", strip=True) if first_td else ""

    # Collect paragraphs of the resolution 
    position_text_parts = [title_text]  # start with title
    for p in position_table.find_all("p"):
        text = p.get_text(" ", strip=True)
        position_text_parts.append(text)

    position_text = "\n".join(position_text_parts)

    return position_text

In [14]:
# Function to get other texts adopted if there are any other adopted texts on the page
def get_other_texts_adopted(url, exclude_titles=['title1', 'title2']):
    """
    Extracts text from all title sections in a DOCEO page, excluding specified titles.
    Returns a dict {title_text: {"text": full_text, "footnotes": {footnote_number: footnote_text}}}
    """
    if exclude_titles is None:
        exclude_titles = []

    response = requests.get(url)
    response.encoding = "utf-8"
    soup = BeautifulSoup(response.text, "html.parser")

    results = {}

    # Find all <a> tags whose "name" attribute starts with "title"
    title_anchors = soup.find_all("a", {"name": lambda x: x and x.startswith("title")})

    # Iterate over each anchor found
    for anchor in title_anchors:
        title_name = anchor.get("name")
        # Skip excluded titles
        if title_name in exclude_titles:
            continue

        # Find the next table with class "doc_box_header" after the current anchor
        table = anchor.find_next("table", class_="doc_box_header")
        if not table:
            continue

        # Extract the title text from the first <td> aligned to the left
        first_td = table.find("td", align="left")
        title_text = first_td.get_text(" ", strip=True) if first_td else title_name

        # Collect all paragraph texts found inside the table
        text_parts = []
        for p in table.find_all("p"):
            text_parts.append(p.get_text(" ", strip=True))
        # Combine all paragraphs into one string separated by line breaks
        full_text = "\n".join(text_parts)

        # Store the extracted text in the results dictionary
        results[title_text] = {
            "text": full_text,
        }
        
    # Return the results dictionary, or None if it is empty
    return results if results != {} else None

In [15]:
# Ensure columns exist before the loop
# Determine the maximum number of URLs in the 'decisions_url_list' column
max_urls = df['decisions_url_list'].apply(lambda x: len(x) if x is not None else 0).max()

# Dynamically create empty columns for each possible decision (resolution, position, and other adopted texts)
for i in range(max_urls):
    df.loc[:, f'resolution_text_{i+1}'] = None
    df.loc[:, f'position_text_{i+1}'] = None
    df.loc[:, f'text_adopted_others_{i+1}'] = None


# Now fill the columns with scraped data
for idx, row in df.iterrows():
    print(f'Processing row {idx} out of {len(df)}')
    # Retrieve the list of decision URLs for the current procedure
    urls = df.loc[idx, 'decisions_url_list']
    # Only proceed if there are URLs available
    if urls is not None:
        for i, url in enumerate(urls):
            # Call functions that scrape different parts of each decision page
            df.loc[idx, f'resolution_text_{i+1}'] = get_resolution_text(url)
            df.loc[idx, f'position_text_{i+1}'] = get_position_text(url)
            df.loc[idx, f'text_adopted_others_{i+1}'] = str(get_other_texts_adopted(url))

# Display the updated dataframe with the extracted text columns
display(df)

Processing row 0 out of 119
Processing row 1 out of 119
Processing row 2 out of 119
Processing row 3 out of 119
Processing row 4 out of 119
Processing row 5 out of 119
Processing row 6 out of 119
Processing row 7 out of 119
Processing row 8 out of 119
Processing row 9 out of 119
Processing row 10 out of 119
Processing row 11 out of 119
Processing row 12 out of 119
Processing row 13 out of 119
Processing row 14 out of 119
Processing row 15 out of 119
Processing row 16 out of 119
Processing row 17 out of 119
Processing row 18 out of 119
Processing row 19 out of 119
Processing row 20 out of 119
Processing row 21 out of 119
Processing row 22 out of 119
Processing row 23 out of 119
Processing row 24 out of 119
Processing row 25 out of 119
Processing row 26 out of 119
Processing row 27 out of 119
Processing row 28 out of 119
Processing row 29 out of 119
Processing row 30 out of 119
Processing row 31 out of 119
Processing row 32 out of 119
Processing row 33 out of 119
Processing row 34 out of

,url,reference,title,subjects,key_players,key_events,documentation_gateway,transparency,final_act,is_legislative_proposal,Procedure reference,Procedure type,Procedure subtype,Legislative instrument,Amendments and repeals,Legal basis,Stage reached in procedure,Committee dossier,Mandatory consultation of other institutions,Other legal basis,legislative_proposal_url,legislative_proposal_text,decisions_url_list,resolution_text_1,position_text_1,text_adopted_others_1
0,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0255(COD),Justice programme 2028-2034,7.40.02 Judicial cooperation in civil and comm...,{'European Parliament': [{'Joint committee res...,{'Legislative proposal published': {'Date': '0...,"[{'Institution': 'European Commission', 'Docum...",NaN,NaN,False,2025/0255(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,"[{'text': 'Repealing Regulation 2021/693'}, {'...",Rules of Procedure EP 58 Treaty on the Functio...,Awaiting committee decision,CJ03/10/03804,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 3.9.2025 COM(2...",None,None,None,None
1,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0405(COD),Placing on the market of genetically modified ...,"3.10.09.06 Agro-genetics, GMOs, 4.60.02 Consum...",{'European Parliament': [{'Joint committee res...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...",NaN,NaN,True,2025/0405(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Directive,"[{'text': 'Amending Directive 2001/18/EC'}, {'...",Treaty on the Functioning of the EU TFEU 114 T...,Preparatory phase in Parliament,NaN,[{'text': 'European Economic and Social Commit...,NaN,NaN,NaN,None,None,None,None
2,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0540(COD),"Union support for asylum, migration and integr...",7.10 Free movement and integration of third-co...,{'European Parliament': [{'Committee responsib...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...",{'Other Members': [{'Name': [{'text': 'ASENS L...,NaN,True,2025/0540(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,NaN,Rules of Procedure EP 58 Treaty on the Functio...,Awaiting committee decision,LIBE/10/03652,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 16.7.2025 COM(...",None,None,None,None
3,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0550(COD),'AgoraEU' programme 2028–2034,"1.20 Citizen's rights, 4.10.03 Child protectio...",{'European Parliament': [{'Joint committee res...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",NaN,True,2025/0550(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,"[{'text': 'Repealing Regulation 2021/818'}, {'...",Rules of Procedure EP 58 Rules of Procedure EP...,Awaiting committee decision,CJ65/10/03702,[{'text': 'European Economic and Social Commit...,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 16.7.2025 COM(...",None,None,None,None
4,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0227(COD),Global Europe,"6.20 Common commercial policy in general, 8.70...",{'European Parliament': [{'Joint committee res...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...",NaN,NaN,True,2025/0227(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,NaN,Rules of Procedure EP 58 Rules of Procedure EP...,Awaiting committee decision,CJ19/10/03697,"[{'text': 'European Court of Auditors', 'href'...",NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 16.7.2025 COM(...",None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [16]:
# Save the data
df.to_csv('final_sample_cod_TA_2025.csv')